In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import scvi

import sys
sys.path.extend(['../../scripts','../../scripts/xenium'])
import readwrite
cfg = readwrite.config()

## Load data

In [2]:
# cfg paths
xenium_dir = Path(cfg['xenium_processed_data_dir'])
results_dir = Path(cfg['results_dir'])

# Segmentation, mapping paths
dir_segmentations = {
    dir_segmentation.name: (dir_segmentation)
    for dir_segmentation in xenium_dir.iterdir()
}

# Read resegmentations and RCTD
xenium_paths = {}
cc_paths = []

for segmentation in (segmentations := xenium_dir.iterdir()):
    for condition in (conditions := segmentation.iterdir()): 
        for panel in (panels := condition.iterdir()):
            for donor in (donors := panel.iterdir()):
                for sample in (samples := donor.iterdir()):
                    
                    k = (segmentation.stem,condition.stem,panel.stem,donor.stem,sample.stem)
                    sample_path = sample / "normalised_results/outs"
                    name = '/'.join(k)

                    xenium_paths[k] = sample_path

xenium_paths = {list(xenium_paths)[0]:xenium_paths[list(xenium_paths)[0]]}

xenium_levels = ('segmentation','condition','panel','donor','sample')
# ads = readwrite.read_xenium_donors(xenium_paths,anndata_only=True,transcripts=False,donor_name_as_key=False)

In [8]:
adata = spatialdata_io.xenium(
    path,
    cells_as_circles=False,
    cells_boundaries=False,
    nucleus_boundaries=False,
    cells_labels=False,
    nucleus_labels=False,
    transcripts=False,
    morphology_mip=False,
    morphology_focus=False,
    aligned_images=False,
)["table"]


INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/segger/breast/br
         east/1FYB/1FYB/normalised_results/outs/cell_feature_matrix.h5                                             


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:158: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)


In [ ]:
ref_adata = ads[list(ads)[0]]

scvi.external.RESOLVI.setup_anndata(ref_adata, labels_key=None, layer=None, prepare_data_kwargs={"spatial_rep": "spatial"})
resolvi = scvi.external.RESOLVI(ref_adata, semisupervised=False)
resolvi.train(max_epochs=50)

donors_corr = resolvi.donor_posterior(
    model=resolvi.module.model_corrected,
    return_sites=["px_rate"],
    summary_fun={"post_donor_q50": np.median},
    num_donors=3,
    summary_frequency=30,
)
donors_corr = pd.DataFrame(donors_corr).T

donors = resolvi.donor_posterior(
    model=resolvi.module.model_residuals,
    return_sites=["mixture_proportions"],
    summary_fun={"post_donor_means": np.mean},
    num_donors=3,
    summary_frequency=100,
)
donors = pd.DataFrame(donors).T

donors_corr = pd.DataFrame(donors_corr.loc["post_donor_q50", "px_rate"],index=ref_adata.obs_names,columns=ref_adata.var_names)
donors_proportions =pd.DataFrame(donors.loc["post_donor_means", "mixture_proportions"],index=ref_adata.obs_names,columns=["true_proportion", "diffusion_proportion", "background_proportion"])

donors_corr.to_parquet(results_dir / "resolvi_corrected.parquet")
donors_proportions.to_parquet(results_dir / "resolvi_proportions.parquet")

INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/resolvi/lib/python3.13/site-packages/scvi/data/fields/_dataframe_field.py:187: UserWarning: Category 0 in adata.obs['_scvi_ind_x'] has fewer than 3 cells. Models may not train properly.
  categorical_mapping = _make_column_categorical(
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/resolvi/lib/python3.13/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/resolvi/lib/python3.13/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but

Epoch 50/50: 100%|██████████| 50/50 [09:01<00:00, 10.92s/it, v_num=1, elbo_train=1.85e+7]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 50/50: 100%|██████████| 50/50 [09:01<00:00, 10.83s/it, v_num=1, elbo_train=1.85e+7]

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/resolvi/lib/python3.13/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...



Sampling local variables, batch: 100%|██████████| 294/294 [00:21<00:00, 13.44it/s]


In [34]:
donors = resolvi.donor_posterior(
    model=resolvi.module.model_residuals,
    return_sites=["mixture_proportions"],
    summary_fun={"post_donor_means": np.mean},
    num_donors=3,
    summary_frequency=100,
)
donors = pd.DataFrame(donors).T

ref_adata.obs[["true_proportion", "diffusion_proportion", "background_proportion"]] = donors.loc[
    "post_donor_means", "mixture_proportions"
]

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/resolvi/lib/python3.13/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/minif ...


Sampling local variables, batch: 100%|██████████| 294/294 [00:18<00:00, 15.51it/s]


,true_proportion,diffusion_proportion,background_proportion
aaabbffd-1,1.000998,0.011002,0.001
aaabhgeo-1,1.000997,0.011003,0.001
aaacokdi-1,1.000996,0.011004,0.001
aaafocon-1,1.000997,0.011003,0.001
aaahgjag-1,1.000997,0.011003,0.001
...,...,...,...
oilhnodh-1,1.000997,0.011003,0.001
oilidbpj-1,1.000997,0.011003,0.001
oilifkch-1,1.000996,0.011004,0.001
oilkgabl-1,1.000997,0.011003,0.001
